$\Delta u + k^2(1+q) u = f $ in $\Omega = [0,1]^2$    
$u = 0 $ on $\partial \Omega$

In [16]:
import scipy as sp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import gmres,spilu,LinearOperator,lgmres
import time
import random

In [17]:
k = 1  # wavenumber
N = 100  # 格点数
h = 1 / N  # 间隔

In [18]:
def q_gen_example(N):
    q = np.zeros((N + 1, N + 1))
    q_value = 0.02
    x1, x2, x3, y1, y2, y3, y4 = 0.2, 0.4, 0.7, 0.2, 0.3, 0.6, 0.7
    q[int(x1 * N):int(x2 * N), int(y1 * N):int(y4 * N)] = q_value
    q[int(x2 * N):int(x3 * N), int(y2 * N):int(y3 * N)] = q_value
    return q

def q_gen_1(N,b1 = 0.3,b2 = 0.6,a1 = 8,a2 = 9,gamma = 1): # 𝑞(𝑥,𝑦)= 𝜆   exp⁡(−𝑎1(𝑥−𝑏1 )^2−a2 (y−b2 )^2 ) 

    q = np.zeros((N + 1, N + 1))
    for i in range(1,N):
        for j in range(1,N):
            q[i,j] = gamma* np.exp(-a1*(i/N - b1)**2-a2*(j/N - b2)**2)
    return q

def q_generation(N, method='T'): 
    if method == 'T':
        return q_gen_example(N)
    elif method == 'Gauss':
        return q_gen_1(N)
    print('method error')


q = q_generation(N)
# sns.heatmap(q, xticklabels=False, yticklabels=False)

$u = \sin(x\pi)\sin(y\pi)$  
$f = \Delta u + (1+q) u = (1+q-2\pi^2)u$

In [19]:
def u_gen(N):
    u = np.zeros((N+1,N+1))
    for i in range(1,N):
        for j in range(1,N):
            u[i,j] = np.sin(i*np.pi/N)*np.sin(j*np.pi/N)
    return u

u_truth = u_gen(N)
# sns.heatmap(u_truth, xticklabels=False, yticklabels=False)

In [20]:
def f_gen_1(N,q,u,k = 1):
    return ((1+q)*k*k-2*np.pi*np.pi)*u

f = f_gen_1(N,q,u_truth)
# sns.heatmap(f, xticklabels=False, yticklabels=False)

###  九点格式
$A \,u_{i,j} = u_{i+1,j} +u_{i-1,j} +u_{i,j+1} +u_{i,j-1}  $  
$\frac{A-4I}{h^2} u_{i,j} + k^2(1+q_{i,j})u_{i,j} = f_{i,j} $ 
$B \,u_{i,j} = u_{i+1,j+1} +u_{i-1,j-1} +u_{i-1,j+1} +u_{i+1,j-1}$  
$\frac{A-4I}{h^2} u_{i,j} + \frac{B - 2A+4I}{6h^2}u_{i,j} + k^2(1+q_{i,j})u_{i,j} + \frac{k^2}{12}(A-4I)
(1+q_{i,j})u_{i,j}= f_{i,j} + \frac{1}{12}(A-4I)f_{i,j}$  
$\Rightarrow\quad (B+4A-20I) u_{i,j} +h^2 k^2(0.5A+4I)
(1+q_{i,j})u_{i,j}= h^2 (0.5A+4I)f_{i,j}$

In [21]:
def A(v):
    v[0, :] = 0
    v[-1, :] = 0
    v[:, 0] = 0
    v[:, -1] = 0
    return v[1:-1, 2:] + v[:-2, 1:-1] + v[2:, 1:-1] + v[1:-1, :-2]


def Matrix_9(N, q, k=1):
    M = N - 1
    h = 1 / N

    Q = q[1:-1, 1:-1].reshape(1, -1)

    value_1 = (1 + Q) * h * h * k * k * 4 - 20  # 主对角线
    value_2 = 4 + 0.5 * h * h * k * k * (1 + q[1:-1, 1:-1])  # A 三对角线&主对角元三对角线

    data_main = value_1  # 主对角线
    data_main_up = np.reshape(np.c_[np.zeros((M, 1)), value_2[:, :-1]],
                              (1, -1))  # 主对角元上对角线
    data_main_down = np.reshape(np.c_[value_2[:, 1:],
                                      np.zeros((M, 1))], (1, -1))  # 主对角元下对角线
    data_up = np.c_[np.zeros((1, M)), value_2[:-1, :].reshape(1,
                                                              -1)]  # 上对角元对角线
    data_up_up = np.tile(np.insert(np.ones(M - 1), 0, 0),
                         M).reshape(1, -1)  # 上对角元上对角线
    data_up_down = np.tile(np.append(np.ones(M - 1), 0),
                           M).reshape(1, -1)  # 上对角元下对角线
    data_down = np.c_[value_2[1:, :].reshape(1, -1), np.zeros((1, M))]  # 下对角元
    data_down_up = data_up_up  # 下对角元上对角线
    data_down_down = data_up_down  # 下对角元下对角线
    data = np.r_[data_main, data_main_up, data_main_down, data_up, data_up_up,
                 data_up_down, data_down, data_down_up, data_down_down]
    offsets = np.array([0, 1, -1, M, M + 1, M - 1, -M, -M + 1, -M - 1])
    dia = sp.sparse.dia_matrix((data, offsets), shape=(M * M, M * M))
    return dia


In [26]:
def perform(N, q_import = False,q_value = 0,q_method = 'T',k = 1,tol=1e-05, restart=20):
    
    def Error(a, a_truth, gap=1e-10):
        a1 = np.where(a < gap, gap, a)
        a_t1 = np.where(a_truth < gap, gap, a_truth)
        return np.abs(a1 / a_t1 - 1)
    
    if q_import==True:
        q = q_value
    else:
        q = q_generation(N,q_method)
    u_truth = u_gen(N)
    f = f_gen_1(N, q, u_truth,k = k)
    h = 1 / N
    M = N-1
    
    
    ###
    time90 = time.process_time()
    Matrix9 = Matrix_9(N, q,k = k)
    Right9 = ((0.5 * A(f) + 4 * f[1:-1, 1:-1]) * h * h).reshape((-1, 1))
    u_res, exit = gmres(Matrix9, Right9, tol=tol, restart=restart)
    time91 = time.process_time()- time90
    ###
    
    
    if exit == 0:
        res9 = np.zeros((N + 1, N + 1))
        res9[1:-1, 1:-1] = u_res.reshape(N - 1, N - 1)
        err9 = np.linalg.norm(Error(res9, u_truth), ord=2) / (N - 1)
    else:
        print('九点格式不收敛')
        return 0
    return err9, time91

# def mean(N, q_method = 'T',k = 1,tol=1e-05, restart=20,iter = 10,q_import= 'False',q_value = 0):
#     err,time_form,time_compute = 0,0,0
#     for i in range(iter):
#         tmp = perform(N, q_method = q_method,k = k,tol=tol, restart=restart,q_import = q_import,q_value = q_value)
#         err+=tmp[0]
#         time+=tmp[1]
#     return err/iter,time/iter

In [8]:
def samp(m,N): #m 样本量
    ll = []
    for i in range(m):
        n = random.randint(1,5)
        q_value = np.zeros((N+1,N+1))
        for j in range(n):
            a1 = random.uniform(5,10)
            a2 = random.uniform(5,10)
            b1 = random.uniform(0.2,0.8)
            b2 = random.uniform(0.2,0.8)
            gamma = random.uniform(-0.5,0.5)
            q_value += q_gen_1(N,b1,b2,a1,a2,gamma)
        ll.append(q_value)
    return ll

In [9]:
N = 100 # 格点数
samples = 1000
%time res = samp(samples,N)
err,time0 = 0,0
for i in range(samples):
    tmp = perform(N, q_import = True,q_value = res[i],k = 2,tol=1e-02, restart=100)
    err+=tmp[0]
    time0+=tmp[1]
    
print(err/samples,time0/samples)

CPU times: user 29.4 s, sys: 90.8 ms, total: 29.5 s
Wall time: 29.6 s
0.003165948445499477 0.03703634300000005


In [10]:
N = 100 # 格点数
samples = 1000
%time res = samp(samples,N)
err,time0 = 0,0
for i in range(samples):
    tmp = perform(N, q_import = True,q_value = res[i],k = 2,tol=1e-04, restart=100)
    err+=tmp[0]
    time0+=tmp[1]

print(err/samples,time0/samples)

CPU times: user 29.7 s, sys: 86.2 ms, total: 29.8 s
Wall time: 29.3 s
2.7862640201586627e-05 0.1220773870000012


In [11]:
N = 50 # 格点数
samples = 1000
%time res = samp(samples,N)
err,time0 = 0,0
for i in range(samples):
    tmp = perform(N, q_import = True,q_value = res[i],k = 2,tol=1e-02, restart=100)
    err+=tmp[0]
    time0+=tmp[1]

print(err/samples,time0/samples)

CPU times: user 7.73 s, sys: 40.6 ms, total: 7.77 s
Wall time: 7.24 s
0.00298540963584686 0.0015491120000008323


In [12]:
N = 50 # 格点数
samples = 1000
%time res = samp(samples,N)
err,time0 = 0,0
for i in range(samples):
    tmp = perform(N, q_import = True,q_value = res[i],k = 2,tol=1e-04, restart=100)
    err+=tmp[0]
    time0+=tmp[1]

print(err/samples,time0/samples)

CPU times: user 7.14 s, sys: 29.6 ms, total: 7.17 s
Wall time: 7.18 s
2.9202493529335928e-05 0.0036029189999989626


In [13]:
N = 200 # 格点数
samples = 100
%time res = samp(samples,N)
err,time0 = 0,0
for i in range(samples):
    tmp = perform(N, q_import = True,q_value = res[i],k = 2,tol=1e-02, restart=100)
    err+=tmp[0]
    time0+=tmp[1]

print(err/samples,time0/samples)

CPU times: user 12.9 s, sys: 64.6 ms, total: 13 s
Wall time: 13.1 s
0.0035830920504443925 0.3622216700000013


In [14]:
N = 200 # 格点数
samples = 100
%time res = samp(samples,N)
err,time0= 0,0
for i in range(samples):
    tmp = perform(N, q_import = True,q_value = res[i],k = 2,tol=1e-04, restart=100)
    err+=tmp[0]
    time0+=tmp[1]

print(err/samples,time0/samples)

CPU times: user 12.7 s, sys: 41.5 ms, total: 12.8 s
Wall time: 12.3 s
6.025622543871892e-05 1.1429803099999987


In [15]:
def perform_pre(N, q_import = False,q_value = 0,q_method = 'T',k = 1,tol=1e-05, restart=20):
    
    def Error(a, a_truth, gap=1e-10):
        a1 = np.where(a < gap, gap, a)
        a_t1 = np.where(a_truth < gap, gap, a_truth)
        return np.abs(a1 / a_t1 - 1)
    
    if q_import==True:
        q = q_value
    else:
        q = q_generation(N,q_method)
    u_truth = u_gen(N)
    f = f_gen_1(N, q, u_truth,k = k)
    h = 1 / N
    M = N-1
    time90 = time.process_time()
    Matrix9 = Matrix_9(N, q,k = k).tocsc()
    Right9 = ((0.5 * A(f) + 4 * f[1:-1, 1:-1]) * h * h).reshape((-1, 1))
    M1 = spilu(Matrix9)
    M_x = lambda x: M1.solve(x)
    M2 = LinearOperator((M*M,M*M), M_x)
    u_res, exit = gmres(Matrix9, Right9, tol=tol, restart=restart,M=M2)
    time91 = time.process_time() - time90
    if exit == 0:
        res9 = np.zeros((N + 1, N + 1))
        res9[1:-1, 1:-1] = u_res.reshape(N - 1, N - 1)
        err9 = np.linalg.norm(Error(res9, u_truth), ord=2) / (N - 1)
    else:
        print('九点格式不收敛')
        return 0


    return err9, time91